In [ ]:
!pip install torch torchvision torchaudio

In [10]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np

# Завантаження датасету
df = pd.read_csv("Life Expectancy Data.csv")

# Видалення рядків із пропусками
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean',fill_value=None)
df_nan = df.isnull().sum()
list_col_to_fill = list(df_nan[df_nan > 0].index)
for cl_n in list_col_to_fill:
    df[cl_n] = imputer.fit_transform(df[[cl_n]])

# Вибір ознак (X) і цільової змінної (y)
# Наприклад, прогнозуємо тривалість життя
X = df.drop(columns=["Life expectancy ", "Country"])
y = df["Life expectancy "]

# Масштабування числових ознак
from sklearn.preprocessing import StandardScaler

# Вибір числових колонок
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Нормалізація
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
# Перевірка типів даних
print("Типи даних:\n", X.dtypes)

# Визначення категоріальних колонок
categorical_columns = X.select_dtypes(include=['object']).columns
print("Категоріальні колонки:\n", categorical_columns)

# Кодування категоріальних колонок
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)
X['Status_Developing'] = X['Status_Developing'].astype(int)
print("Розмір після кодування:", X.shape)



# Розділення на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Перетворення в тензори PyTorch
X_train = torch.tensor(X_train.values, dtype=torch.float32)  # Додаємо .values
X_test = torch.tensor(X_test.values, dtype=torch.float32)    # Додаємо .values
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Архітектура повнозв'язаної нейронної мережі
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Ініціалізація моделі
input_dim = X_train.shape[1]
model = FeedForwardNN(input_dim)

# Вибір оптимізатора та функції втрат
criterion = nn.MSELoss()  # Оскільки це регресійна задача
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Навчання моделі
num_epochs = 1000
for epoch in range(num_epochs):
    # Прогноз
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Оновлення ваг
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Оцінка моделі
with torch.no_grad():
    y_pred_test = model(X_test)
    test_loss = criterion(y_pred_test, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

from sklearn.metrics import r2_score

# Оцінка моделі
with torch.no_grad():
    y_pred_test = model(X_test).numpy()  # Передбачення для тестових даних
    y_test_numpy = y_test.numpy()  # Перетворення тестових значень у numpy

    # Розрахунок R^2
    r2 = r2_score(y_test_numpy, y_pred_test)
    print(f"Test R^2 Score: {r2:.4f}")

Типи даних:
 Year                               float64
Status                              object
Adult Mortality                    float64
infant deaths                      float64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                            float64
 BMI                               float64
under-five deaths                  float64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
GDP                                float64
Population                         float64
 thinness  1-19 years              float64
 thinness 5-9 years                float64
Income composition of resources    float64
Schooling                          float64
dtype: object
Категоріальні колонки:
 Index(['Status'], dtype='object')
Розмір після кодування: (2938, 20)
Epoch 10/1000, Loss:

In [11]:
## 2 

In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# Шлях до папки з даними
data_path = "tom_and_jerry_dataset"

# Перевірка структури датасету
print("Класи в датасеті:", os.listdir(data_path))

# Підготовка трансформацій
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Зміна розміру зображень
    transforms.ToTensor(),  # Перетворення в тензор
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Нормалізація
])

# Завантаження даних
dataset = ImageFolder(root=data_path, transform=transform)

# Розділення даних на тренувальні та тестові
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Завантаження в DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Перевірка класів
print(f"Кількість класів: {len(dataset.classes)}, Класи: {dataset.classes}")
